In [1]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras import Sequential
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

In [2]:
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02) #standardna devijacija
    src_image = Input(shape=image_shape)
    target_image = Input(shape=image_shape)
    ## razmisliti da li ovdde da spajam ili ne, slike?, i da li mi to treba
    model = Sequential()
    model.add(Conv2D(64, (4, 4), strides=(2,2), padding='same', kernel_initializer=init))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (4,4), padding='same', kernel_initializer=init))
    model.add(Activation('sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [3]:
def encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [4]:
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, skip_in])
    g = Activation('relu')(g)
    return g

In [5]:
def define_generator(image_shape=(256,256,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    e1 = encoder_block(in_image, 64, batchnorm=False)
    e2 = encoder_block(e1, 128)
    e3 = encoder_block(e2, 256)
    e4 = encoder_block(e3, 512)
    e5 = encoder_block(e4, 512)
    e6 = encoder_block(e5, 512)
    e7 = encoder_block(e6, 512)
   
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
   
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model

In [6]:
def gan(generator_model, discriminator_model, image_shape):
    for layer in discriminator_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    in_src = Input(shape=image_shape)
    
    gen_out = generator_model(in_src)
    dis_out = discriminator_model([in_src, gen_out])
    model = Model(in_src, [dis_out, gen_out])
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
def load_real_samples(filename):
    data = load(filename) #todo 
    X1, X2 = data['arr_0'], data['arr_1']
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

In [8]:
def generate_real_samples(dataset, n_samples, patch_shape):
    trainA, trainB = dataset
    ix = randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[ix], trainB[ix]
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y

In [9]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
    
    n_patch = d_model.output_shape[1]
   
    trainA, trainB = dataset
    
    bat_per_epo = int(len(trainA) / n_batch)
    
    n_steps = bat_per_epo * n_epochs
   
    for i in range(n_steps):
       
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
       
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
       
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
       
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))

In [15]:
dataset = make_npz()  
print(dataset["x"]) #todo uraditi resize slika :( 
#dataset = load_real_samples('maps_256.npz')
#print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = (1,256,128)#128) #dataset[0].shape[1:]
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model, dataset)

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x2852D256130>
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x2852D2564C0>
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x2852D256280>
 ...
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x28537172460>
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x285371723A0>
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x128 at 0x28537172DC0>]


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 2, 10, 512), (None, 1, 10, 512)]

In [11]:
from tempfile import TemporaryFile
import numpy as np
import os
from PIL import Image

In [12]:
def make_npz():
    outfile = TemporaryFile()
    x = []
    for subdir, dirs, files in os.walk("C:/soft/bb/MusicGenerator/images"):
        for file in files:
            img = Image.open(os.path.join(subdir, file))
            x.append(img)
            
    np.savez(outfile, x)
    _ = outfile.seek(0)
    npzfile = np.load(outfile)
    npzfile.files
    outfile = TemporaryFile()
    np.savez(outfile, x=x)
    _ = outfile.seek(0)
    npzfile = np.load(outfile, allow_pickle=True)
    return npzfile